In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('../../raw/CarsSaleForecast/[new] yancheng_train_20171226.csv')
test = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')
trainFeature = pd.read_csv('../../raw/LiChuan/train_feature.csv')

In [3]:
trainDate = train[['sale_date', 'sale_quantity', 'class_id']]
trainDate = trainDate.groupby(['sale_date', 'class_id']).agg('sum').reset_index()
trainDate['year'] = trainDate['sale_date'].apply(lambda x: int(str(x)[0:4]))
trainDate['month'] = trainDate['sale_date'].apply(lambda x: int(str(x)[4:6]))
test['year'] = test['predict_date'].apply(lambda x: int(str(x)[0:4]))
test['month'] = test['predict_date'].apply(lambda x: int(str(x)[4:6]))

In [4]:
# 2012.1 data
trainSaleDate = trainFeature
trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['month']==1)]
trainTmp.loc[:, 'how_many_month_has_sell'] = 1
sale_class_count = len(trainTmp)

trainSaleDate = pd.merge(trainSaleDate, trainTmp, on='class_id', how='left')
trainSaleDate['year'].fillna(2012.0, inplace=True)
trainSaleDate['month'].fillna(1.0, inplace=True)
trainSaleDate.loc[:, 'sale_quantity_total_last_month'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_month'] = 0
trainSaleDate.loc[:, 'this_month_sale_class_count'] = sale_class_count

In [5]:
# 2012.2 - 2012.12 data
for month in range(2, 13):
    trainSaleDateTmp = trainFeature
    trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['month']==month)]
    
    sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==2012)&(trainSaleDate['month']==(month-1))]
    sellmonth.dropna(inplace=True)
    sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
    sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
    sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)
    
    trainTmp = pd.merge(trainTmp, sellmonth, on='class_id', how='left')
    trainTmp['how_many_month_has_sell'].fillna(1.0, inplace=True)
    sale_class_count = len(trainTmp)
    
    trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
    trainSaleDateTmp['year'].fillna(2012.0, inplace=True)
    trainSaleDateTmp['month'].fillna(month, inplace=True)
    trainSaleDateTmp.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
    trainSaleDateTmp.loc[:, 'this_month_sale_class_count'] = sale_class_count
    
    trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
    
trainSaleDate.loc[:, 'sale_quantity_total_last_year'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_year'] = 0

In [6]:
# 2013-2016
for year in range(2013, 2018):
    for month in range(1, 13):
        trainSaleDateTmp = trainFeature
        trainTmp = trainDate[(trainDate['year']==year)&(trainDate['month']==month)]
        
        if month == 1:
            sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==(year-1))&(trainSaleDate['month']==12)]
        else:
            sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==year)&(trainSaleDate['month']==(month-1))]
        sellmonth.dropna(inplace=True)
        sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
        sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
        sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)
        
        sellYear = trainSaleDate[['class_id', 'sale_quantity']][((trainSaleDate['year']==(year-1))&(trainSaleDate['month']>=month))|((trainSaleDate['year']==year)&(trainSaleDate['month']<month))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellmonth, on='class_id', how='left')
        trainTmp['how_many_month_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, on='class_id', how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['month'].fillna(month, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_month_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])

In [7]:
trainSaleDate = trainSaleDate[:-280]
trainSaleDate.sort_values(['year', 'month', 'class_id'])
trainSaleDate.fillna(0.0, inplace=True)

In [8]:
sellmonth = trainSaleDate[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(trainSaleDate['year']==2017)&(trainSaleDate['month']==10)]
sellmonth.dropna(inplace=True) 
sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)

sellYear = trainSaleDate[['class_id', 'sale_quantity']][((trainSaleDate['year']==2016)&(trainSaleDate['month']>=11))|((trainSaleDate['year']==2017)&(trainSaleDate['month']<11))]
sellYear.dropna(inplace=True)
sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)

test = pd.merge(test, trainFeature, on='class_id', how='left')
test = pd.merge(test, sellmonth, on='class_id', how='left')
test = pd.merge(test, sellYear, on='class_id', how='left')

test.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
test.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
test.loc[:, 'this_month_sale_class_count'] = 140

test.sort_values(['class_id'])
test.fillna(0.0, inplace=True)

test = test.drop(['predict_quantity'], axis=1)

In [9]:
columns = train.drop(['sale_quantity'], axis=1).columns.tolist()
test['sale_date'] = test['predict_date']
test = test.drop(['predict_date'], axis=1)

In [11]:
trainSaleDate.to_csv('../../raw/LiChuan/trainSaleDatetest.csv', index=False)
test.to_csv('../../raw/LiChuan/test_featuretest.csv', index=False)